In [4]:
'''
Họ và tên: Nguyễn Thế Vỹ MSSV: 22200186

Báo cáo bài tập: Môn Thực hành Nhập môn Xử lý ảnh và Video
'''

'\nHọ và tên: Nguyễn Thế Vỹ MSSV: 22200186\n\nBáo cáo bài tập: Môn Thực hành Nhập môn Xử lý ảnh và Video\n'

In [ ]:
'''Câu 1. Viết chương trình thực hiện DCT thay cho DCT từ thư viện fft.'''
import numpy as np

def get_epsilon(k):
    return 1 / np.sqrt(2) if k == 0 else 1

def dct2(block):
    """Manual 2D DCT for 8x8 block."""
    N = 8
    result = np.zeros((N, N))
    for u in range(N):
        for v in range(N):
            sum_val = 0.0
            for x in range(N):
                for y in range(N):
                    cos_x = np.cos((2 * x + 1) * u * np.pi / 16)
                    cos_y = np.cos((2 * y + 1) * v * np.pi / 16)
                    sum_val += block[x, y] * cos_x * cos_y
            result[u, v] = (get_epsilon(u) * get_epsilon(v) / 4) * sum_val
    return result

def idct2(block):
    """Manual 2D inverse DCT for 8x8 block."""
    N = 8
    result = np.zeros((N, N))
    for x in range(N):
        for y in range(N):
            sum_val = 0.0
            for u in range(N):
                for v in range(N):
                    cos_x = np.cos((2 * x + 1) * u * np.pi / 16)
                    cos_y = np.cos((2 * y + 1) * v * np.pi / 16)
                    sum_val += get_epsilon(u) * get_epsilon(v) * block[u, v] * cos_x * cos_y
            result[x, y] = (1 / 4) * sum_val  # Normalize by 1/4
    return result


#test với random ma trận 8x8
block = np.random.randint(0, 256, (8, 8))
print("random block: \n", block)
print("\n======================================\n")
dct_block = dct2(block)
print("DCT block:\n", dct_block)


random block: 
 [[201 198 157  20  67  50  64 206]
 [ 45 189  65 238 167  13 203  27]
 [242  74 107 164 111  71  47  75]
 [ 32  45 245  48 137 189  42 105]
 [  6 226  22 141 113  66 215  97]
 [ 67 143 199 139  73 115   3  82]
 [ 89  52 143  58 206 213 133 191]
 [238 209  47  21 116 163  18  69]]


DCT block:
 [[ 9.14625000e+02  7.69602191e+01 -7.94665547e+00 -6.57217492e+00
  -1.68750000e+01 -8.46240821e+01 -2.52624561e+00  5.32761041e+00]
 [-6.74824005e-01  4.80436517e+01  2.00519179e+01 -9.42398669e+01
   4.12766700e+01  7.71675551e+01 -9.06350650e+01  2.68472774e+01]
 [ 4.04296866e+01  7.37045907e+01  1.40168340e+02  7.36115702e+01
   5.33705991e+01 -5.09910096e+01  3.45197955e+00  6.12292790e+00]
 [ 8.89850097e+00 -5.11170489e+01  2.82912381e+01 -1.24753541e+02
  -6.28054615e+01 -3.99204909e+01 -4.70036075e+01 -1.15464682e+01]
 [-2.13750000e+01  9.28261594e+00  1.13552435e+02  2.20234418e+01
  -6.78750000e+01 -9.23963071e+01  1.64202841e+01 -6.41079277e-01]
 [ 2.80735989e+01 -6.960

In [30]:
'''Câu 2
Thuật toán tính nhanh DCT (Bin DCT) - 
Đọc hiểu và giải thích: sơ đồ thuật toán Bin DCT loại A, B, C  (Hình 4,5,6 trong file binDCT1.pdf) và thuật toán chung của BinDCT (có hệ số scale) (hình 5 của file binDCT.pdf) - 
Viết code thực hiện BinDCT loại C thay cho DCT ở câu 1. 

'''


'''
Giải thích sơ đồ thuật toán Bin DCT loại A, B, C và thuật toán chung của BinDCT (có hệ số scale):
Dựa trên nội dung các file PDF:

BinDCT loại A, B, C (Hình 4,5,6 trong binDCT1.pdf):
Đây là các xấp xỉ nhanh của DCT sử dụng chỉ shift (>> hoặc <<) và add (+/-), không multiply, dựa trên lifting scheme (ladder structure) để map integer-to-integer, phù hợp VLSI và lossless.

--> Cụ thể ở trong từng loại thì:

Loại A (Hình 4): Flowgraph đơn giản nhất, xấp xỉ DCT với 14 shifts và 31 adds cho 8 samples. Bắt đầu từ input x[0] đến x[7], qua các butterflies (add/sub), sau đó lifting steps với coefficients dyadic (như 1/2, 1/4). Output X[0], X[4],... Có scale factor (hệ số scale) ở cuối để normalize. Coding gain ~8.77 dB, gần DCT thật (8.82 dB). Sử dụng cho low complexity.
Loại B (Hình 5): Cải tiến từ A, thêm lifting steps để tăng accuracy. 16 shifts và 31 adds. Coding gain cao hơn (~8.80 dB). Flowgraph có thêm rotations xấp xỉ (như π/4 ≈ 1/2 + 1/4). Phù hợp medium complexity.
Loại C (Hình 6): Phức tạp nhất trong 3 loại, 18 shifts và 31 adds. Coding gain gần DCT nhất (~8.82 dB). Flowgraph có nhiều lifting steps hơn, xấp xỉ tốt các góc rotation như 3π/16, 7π/16 từ DCT gốc (Hình 1 binDCT1.pdf). Sử dụng cho high accuracy với vẫn chỉ shift/add.

Thuật toán chung của BinDCT (có hệ số scale) (Hình 5 của binDCT.pdf):
Đây là scaled DCT (không normalize ngay, scale factor 1/sqrt(8) hoặc 1/8 ở cuối). Dựa trên factorization của Loeffler (11 multiplies gốc), nhưng thay bằng lifting để loại multiply.
Flowgraph bao gồm butterflies và plane rotations decomposed thành 3 lifting steps mỗi (P, U, S). Coefficients là dyadic rationals (2^{-k}). Tổng 5 rotations → 15 lifting steps. Coding gain từ 8.77-8.82 dB tùy config. Phù hợp JPEG/H.263 vì tích hợp quantization vào scale. Inverse dễ invert bằng reverse lifting.


Tổng quát: BinDCT xấp xỉ DCT bằng lifting để nhanh, lossless, low power. Từ DCT gốc (floating-point multiply), decompose rotations thành lifting: rotation(θ) ≈ [1 p 0; 0 1] [1 0 u; 0 1] [1 s 0; 0 1], với p,u,s ≈ tan(θ/2), sin(θ),... dyadic.

'''

'''
Code thực hiện BinDCT loại C thay cho DCT ở câu 1.
'''
import numpy as np

def bin_dct_1d(x):
    """1D BinDCT Type C forward transform (integer approximation). Input: 8-element array (int)."""
    #đảm bảo x là np.int32 --> đảm bảo đúng chính xác trong các phép dịch bit 
    #bước 1: butterfly sums/diffs
    tmp0 = x[0] + x[7]; tmp7 = x[0] - x[7]
    tmp1 = x[1] + x[6]; tmp6 = x[1] - x[6]
    tmp2 = x[2] + x[5]; tmp5 = x[2] - x[5]
    tmp3 = x[3] + x[4]; tmp4 = x[3] - x[4]

    #bước 2: Scaled sums (3/8, 5/8 xấp xỉ với *3>>3, *5>>3)
    tmp16 = ((tmp5 * 3) >> 3) + tmp6
    tmp15 = ((tmp16 * 5) >> 3) - tmp5

    #bước 3: butterfly sums/diffs
    tmp10 = tmp0 + tmp3; tmp13 = tmp0 - tmp3
    tmp11 = tmp1 + tmp2; tmp12 = tmp1 - tmp2
    tmp14 = tmp4 + tmp15; tmp15_ = tmp4 - tmp15  # Preserve tmp15
    z = tmp16
    tmp16_ = tmp7 - tmp16
    tmp17 = z + tmp7

    #bước 4: thêm lifting steps với các coefficients xấp xỉ
    tmp14 = (tmp17 >> 3) - tmp14
    tmp10 = tmp10 + tmp11
    tmp11 = (tmp10 >> 1) - tmp11
    tmp12 = ((tmp13 * 3) >> 3) - tmp12
    tmp13 = ((tmp12 * 3) >> 3) + tmp13
    tmp15 = ((tmp16_ * 7) >> 3) + tmp15_
    tmp16_ = (tmp15 >> 1) - tmp16_

    #đầu ra
    out = np.zeros(8, dtype=np.int32)
    out[0] = tmp10
    out[4] = tmp11
    out[6] = tmp12
    out[2] = tmp13
    out[7] = tmp14
    out[5] = tmp15
    out[3] = tmp16_
    out[1] = tmp17
    return out

def bin_dct2(block):
    """2D BinDCT Type C: Apply 1D on rows then columns, with final /8 scale."""
    #đặt kiểu dữ liệu np.int32 để đảm bảo tính chính xác và tránh tràn số
    rows = np.zeros((8, 8), dtype=np.int32)
    for i in range(8):
        rows[i] = bin_dct_1d(block[i])

    out = np.zeros((8, 8), dtype=np.int32)
    for j in range(8):
        col = bin_dct_1d(rows[:, j])
        out[:, j] = col >> 3  #đầu ra cuối ==> sclae cho 8
    return out

ran_block = np.random.randint(0, 256, (8, 8))
print("random block: \n", ran_block)
print("\n======================================\n")
dct_block = bin_dct2(ran_block)
print("BinDCT loại C block:\n", dct_block)



random block: 
 [[220 157 238 188 199 215 158 182]
 [197   7 247  71 196  30 180  79]
 [238 104 194  89 209 137  63 205]
 [204 133 190 251  68 145  97  93]
 [172  38 129 145  35 180 119 246]
 [ 23  68  33 134 131  32  88 239]
 [ 76  26  39 163  77  82 247  98]
 [ 83  70 212  35  86 172 201 115]]


BinDCT loại C block:
 [[1072  -62   79   11   32  124   82   54]
 [ 120   75   26  -10   24  -18   48   46]
 [  48   -9   -8  -13  -25  -79    2   51]
 [   9   41    9  -11  -17   13   22   31]
 [  60   16   27    4  -25    8   36  -39]
 [  68   27  -65   -3    7   12  -52  -85]
 [  34    7   27   12   16 -135   23  -12]
 [  31  -24   54  -17    2   -9   32   -7]]


In [22]:
'''Câu 3: Viết chương trình thực hiện zigzag thay cho phương pháp đang dùng trong demo (không dùng hàm có sẵn). '''


"code:"


def zigzag_scan(block):
    """Manual zigzag scan for 8x8 block, return 1D array."""
    zig_order = [
        (0,0), (0,1), (1,0), (2,0), (1,1), (0,2), (0,3), (1,2),
        (2,1), (3,0), (4,0), (3,1), (2,2), (1,3), (0,4), (0,5),
        (1,4), (2,3), (3,2), (4,1), (5,0), (6,0), (5,1), (4,2),
        (3,3), (2,4), (1,5), (0,6), (0,7), (1,6), (2,5), (3,4),
        (4,3), (5,2), (6,1), (7,0), (7,1), (6,2), (5,3), (4,4),
        (3,5), (2,6), (1,7), (2,7), (3,6), (4,5), (5,4), (6,3),
        (7,2), (7,3), (6,4), (5,5), (4,6), (3,7), (4,7), (5,6),
        (6,5), (7,4), (7,5), (6,6), (5,7), (6,7), (7,6), (7,7)
    ]
    return np.array([block[pos[0], pos[1]] for pos in zig_order])

#Test
block = np.arange(64).reshape(8,8)

print("Block:\n", block)

print("\n======================================\n")
zig = zigzag_scan(block)
print("Zigzag:", zig)

Block:
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]


Zigzag: [ 0  1  8 16  9  2  3 10 17 24 32 25 18 11  4  5 12 19 26 33 40 48 41 34
 27 20 13  6  7 14 21 28 35 42 49 56 57 50 43 36 29 22 15 23 30 37 44 51
 58 59 52 45 38 31 39 46 53 60 61 54 47 55 62 63]


In [35]:
import numpy as np
import pandas as pd
from PIL import Image

original = np.array(Image.open('img1.ppm').convert('L'))

# Câu 1: DCT thủ công
def get_epsilon(k):
    return 1 / np.sqrt(2) if k == 0 else 1

def dct2(block):
    N = block.shape[0]
    result = np.zeros((N, N))
    for u in range(N):
        for v in range(N):
            sum_val = 0.0
            for x_ in range(N):
                for y_ in range(N):
                    cos_x = np.cos((2 * x_ + 1) * u * np.pi / (2 * N))
                    cos_y = np.cos((2 * y_ + 1) * v * np.pi / (2 * N))
                    sum_val += block[x_, y_] * cos_x * cos_y
            result[u, v] = (2 / N) * get_epsilon(u) * get_epsilon(v) * sum_val
    return result

def idct2(block):
    N = block.shape[0]
    result = np.zeros((N, N))
    for x in range(N):
        for y in range(N):
            sum_val = 0.0
            for u in range(N):
                for v in range(N):
                    cos_x = np.cos((2 * x + 1) * u * np.pi / (2 * N))
                    cos_y = np.cos((2 * y + 1) * v * np.pi / (2 * N))
                    sum_val += get_epsilon(u) * get_epsilon(v) * block[u, v] * cos_x * cos_y
            result[x, y] = (2 / N) * sum_val
    return result

# Câu 2: BinDCT loại C
def bin_dct_1d(x):
    tmp0 = x[0] + x[7]; tmp7 = x[0] - x[7]
    tmp1 = x[1] + x[6]; tmp6 = x[1] - x[6]
    tmp2 = x[2] + x[5]; tmp5 = x[2] - x[5]
    tmp3 = x[3] + x[4]; tmp4 = x[3] - x[4]
    tmp16 = ((tmp5 * 3) >> 3) + tmp6
    tmp15 = ((tmp16 * 5) >> 3) - tmp5
    tmp10 = tmp0 + tmp3; tmp13 = tmp0 - tmp3
    tmp11 = tmp1 + tmp2; tmp12 = tmp1 - tmp2
    tmp14 = tmp4 + tmp15; tmp15_ = tmp4 - tmp15
    z = tmp16
    tmp16_ = tmp7 - tmp16
    tmp17 = z + tmp7
    tmp14 = (tmp17 >> 3) - tmp14
    tmp10 = tmp10 + tmp11
    tmp11 = (tmp10 >> 1) - tmp11
    tmp12 = ((tmp13 * 3) >> 3) - tmp12
    tmp13 = ((tmp12 * 3) >> 3) + tmp13
    tmp15 = ((tmp16_ * 7) >> 3) + tmp15_
    tmp16_ = (tmp15 >> 1) - tmp16_
    out = np.zeros(8, dtype=np.int32)
    out[0] = tmp10
    out[4] = tmp11
    out[6] = tmp12
    out[2] = tmp13
    out[7] = tmp14
    out[5] = tmp15
    out[3] = tmp16_
    out[1] = tmp17
    return out

def bin_dct2(block):
    rows = np.zeros((8, 8), dtype=np.int32)
    for i in range(8):
        rows[i] = bin_dct_1d(block[i])
    out = np.zeros((8, 8), dtype=np.int32)
    for j in range(8):
        col = bin_dct_1d(rows[:, j])
        out[:, j] = col >> 3
    return out

# Câu 3: Zigzag
zig_order = [
    (0,0), (0,1), (1,0), (2,0), (1,1), (0,2), (0,3), (1,2),
    (2,1), (3,0), (4,0), (3,1), (2,2), (1,3), (0,4), (0,5),
    (1,4), (2,3), (3,2), (4,1), (5,0), (6,0), (5,1), (4,2),
    (3,3), (2,4), (1,5), (0,6), (0,7), (1,6), (2,5), (3,4),
    (4,3), (5,2), (6,1), (7,0), (7,1), (6,2), (5,3), (4,4),
    (3,5), (2,6), (1,7), (2,7), (3,6), (4,5), (5,4), (6,3),
    (7,2), (7,3), (6,4), (5,5), (4,6), (3,7), (4,7), (5,6),
    (6,5), (7,4), (7,5), (6,6), (5,7), (6,7), (7,6), (7,7)
]

def zigzag_scan(block):
    return np.array([block[pos[0], pos[1]] for pos in zig_order])

def inverse_zigzag_scan(zig):
    inv_block = np.zeros((8,8))
    for k, pos in enumerate(zig_order):
        inv_block[pos[0], pos[1]] = zig[k]
    return inv_block

# Bảng lượng tử
quant_luma = np.array([
    [16, 11, 10, 16, 124, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])

def get_quant_table(quality):
    if quality < 50:
        s = 5000 / quality
    else:
        s = 200 - 2 * quality
    return np.floor((s * quant_luma + 50) / 100).clip(1, 255)

# Simulate nén/giải nén (dựa 1,2,3)
def simulate_compress_decompress(image, dct_func, idct_func, quality):
    h, w = image.shape
    pad_h = (8 - h % 8) % 8
    pad_w = (8 - w % 8) % 8
    padded_image = np.pad(image, ((0, pad_h), (0, pad_w)), mode='edge')
    h_p, w_p = padded_image.shape
    quant = get_quant_table(quality)
    recon_p = np.zeros_like(padded_image, dtype=float)
    for i in range(0, h_p, 8):
        for j in range(0, w_p, 8):
            block = padded_image[i:i+8, j:j+8]
            if dct_func == bin_dct2:
                block = block.astype(np.int32) - 128
            else:
                block = block.astype(float) - 128
            dct_block = dct_func(block)
            zig = zigzag_scan(dct_block)
            q_zig = np.round(zig / quant.flatten())
            dq_zig = q_zig * quant.flatten()
            dq_block = inverse_zigzag_scan(dq_zig)
            idct_block = idct_func(dq_block) + 128
            recon_p[i:i+8, j:j+8] = np.clip(idct_block, 0, 255)
    recon = recon_p[:h, :w].astype(np.uint8)
    return recon

def psnr(original, reconstructed):
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:
        return float('inf')
    max_val = 255.0
    return 10 * np.log10(max_val**2 / mse)

qualities = [95, 80, 50, 20]
results = []
for q in qualities:
    recon_dct = simulate_compress_decompress(original, dct2, idct2, q)
    psnr_dct = psnr(original, recon_dct)
    recon_bin = simulate_compress_decompress(original, bin_dct2, idct2, q)
    psnr_bin = psnr(original, recon_bin)
    results.append((q, round(psnr_dct, 2), round(psnr_bin, 2)))

df = pd.DataFrame(results, columns=['Quality', 'PSNR DCT Thủ công', 'PSNR BinDCT C'])
print(df)

KeyboardInterrupt: 